In [1]:
import pandas as pd
import numpy as np
import re
import plotly as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)
import plotly.io as pio
%config InlineBackend.figure_format = 'svg'

trebleNames=pd.read_csv("../data/TrebleNames.csv")
trebleNames=trebleNames.iloc[:,:4]
trebleNames.columns=['Model','codename','imagType','arch']
trebleNames.drop(columns=['codename','imagType','arch'],inplace=True)
trebleNames.Model=trebleNames.Model.str.strip()
trebleNames.Model=trebleNames.Model.str.lower()
trebleNames["supported"]="yes"
trebleNames=trebleNames.drop_duplicates()

# print(trebleNames.head())

# Normalize the model names
trebleNames.Model=trebleNames.Model.str.replace("samsung ","")
dups=trebleNames[trebleNames.Model.str.contains(" and ")]
for index,value in dups.iterrows():
    parsedManufactures=value[0].split(" ")[0]
    phones = value[0].split(" and ")
    secndPhone=parsedManufactures + " " + phones[1]
    newData=[[secndPhone,value[1]]]
    newPhone=pd.DataFrame(newData.copy(),columns=['Model', 'supported'])
    trebleNames=trebleNames.append(newPhone.copy())
    
trebleNames.reset_index(drop=True,inplace=True)    

trebleNames.Model=trebleNames.Model.str.replace(" and .*$","")
trebleNames.Model=trebleNames.Model.str.replace('nokia v',"nokia 2.1 v")
trebleNames.Model=trebleNames.Model.str.replace('nokia c',"nokia 3.1 c")
trebleNames.Model=trebleNames.Model.str.replace(' \(exynos\)',"")
trebleNames.Model=trebleNames.Model.str.replace(' \(snapdragon\)',"")


dups2=trebleNames[trebleNames.Model.str.contains("galaxy")]
for index,value in dups2.iterrows():
    if " plus" in str(value.Model):
        newVal=value.Model.replace(" plus",'+')
        trebleNames.at[index,'Model']=newVal

   
print(trebleNames.Model.unique())

trebleNames.Model=trebleNames.Model.str.replace('google ',"")
trebleNames.Model=trebleNames.Model.str.replace('lg g7 thinq™️',"lg g7 thinq")
trebleNames.Model=trebleNames.Model.str.replace('lg v30',"v30")

trebleNames=trebleNames.drop_duplicates()

['alcatel 5v 5060d' 'alldocube m5' 'alldocube power m3' 'allview v3 viper'
 'asus rog phone (zs600kl)' 'asus rog phone ii (zs660kl)'
 'asus zenfone 5 (ze620kl)' 'asus zenfone 5z (zs620kl/zs621kl)'
 'asus zenfone go' 'asus zenfone max m1 (zb555kl)'
 'asus zenfone max m1 (zb556kl)' 'asus zenfone max pro m1'
 'asus zenfone max pro m2' 'barnes & noble nook tablet 10.1'
 'blackview a20' 'blackview a60' 'blackview a60 pro' 'blackview bv9500'
 'blackview bv9500 pro' 'blackview bv9600 pro' 'blu vivo xl4'
 'bq aquaris c' 'chuwi hi9 air' 'coolpad c558' 'coolpad cool changer 1c'
 'coolpad cool changer s1' 'cubot p20' 'dexp a140' 'dexp a240' 'dexp b260'
 'dexp bl250' 'doogee x5' 'doogee x5 max' 'doogee x5 pro' 'doogee x50'
 'doogee y8' 'elephone soldier' 'elephone u pro' 'essential ph-1'
 'f(x)tec pro1' 'general mobile gm 5' 'general mobile gm 5 plus'
 'general mobile gm 8' 'general mobile gm 8 go' 'general mobile gm 9 pro'
 'gome u9' 'google pixel' 'google pixel 2' 'google pixel 3'
 'google pixel

In [2]:
carrierData=pd.read_csv("../data/allCarrierData_final.csv")
carrierData["ModelNorm"]=carrierData.Model.str.strip()
carrierData["ModelNorm"]=carrierData.ModelNorm.str.lower()
    

# remove outliers!!
carrierData=carrierData[carrierData.difference>-10]

#merge...
testing=pd.merge(carrierData,trebleNames,left_on=['ModelNorm'],right_on=['Model'],how='outer')
print(testing.head())

  Carrier Manufacture             Model_x Release_Date Bulletin_Level  \
0    AT&T     Alcatel  Alcatel IdealXCITE   2018-04-03     2018-02-01   
1    AT&T     Alcatel  Alcatel IdealXCITE   2018-04-03     2018-04-01   
2    AT&T     Alcatel  Alcatel IdealXCITE   2018-10-24     2018-08-01   
3    AT&T     Alcatel  Alcatel IdealXCITE   2018-12-18     2018-10-01   
4    AT&T  BlackBerry   BlackBerry KEYone   2017-09-26     2017-09-01   

   difference Android_Level_Guessed  \
0        61.0                 False   
1         2.0                  True   
2        84.0                 False   
3        78.0                 False   
4        25.0                 False   

                                               build  Year_Phone_Release  \
0        JO.2.0.c1-00150-8909_GEN_PACK-1 nan YAMUAM0              2017.0   
1                                nan YB4UA40File nan              2017.0   
2        JO.2.0.c1-00267-8909_GEN_PACK-2 nan YAYUAY0              2017.0   
3        JO.2.0.c1-002

In [3]:
testing['supported']=testing.supported.astype("str")
testing['Model_x']=testing.Model_x.astype("str")
testing['Model_y']=testing.Model_y.astype("str")
testing['ModelNorm']=testing.ModelNorm.astype("str")



finalTrebleList=testing[testing.Model_x!='nan']
finalTrebleList=finalTrebleList[finalTrebleList.supported=='yes']

nonTrebleList=testing[testing.Model_x!='nan']
nonTrebleList=nonTrebleList[nonTrebleList.supported=='nan']

finalTrebleList.drop(columns=['ModelNorm','Model_y','supported'],inplace=True)
nonTrebleList.drop(columns=['ModelNorm','Model_y','supported'],inplace=True)


print(finalTrebleList.head())
print("---------------------------------------------------------------------------")
print(nonTrebleList.head())


    Carrier Manufacture Model_x Release_Date Bulletin_Level  difference  \
221    AT&T          LG     V30   2017-10-06     2017-10-01         5.0   
222    AT&T          LG     V30   2017-12-19     2017-12-01        18.0   
223    AT&T          LG     V30   2018-02-23     2018-01-01        53.0   
224    AT&T          LG     V30   2018-08-06     2018-05-01        97.0   
225    AT&T          LG     V30   2019-11-19     2019-11-01        18.0   

    Android_Level_Guessed            build  Year_Phone_Release  \
221                 False  nan nan H93120d              2017.0   
222                  True      nan nan nan              2017.0   
223                 False  nan nan H93111N              2017.0   
224                 False  nan nan H93120c              2017.0   
225                  None  nan nan H93120d              2017.0   

     Month_Phone_Release  checker  
221                  9.0      1.0  
222                  9.0      1.0  
223                  9.0      5.0  
224     

In [4]:
print(len(carrierData))
print(len(trebleNames))
print(len(testing))
print(len(finalTrebleList))
print(len(nonTrebleList))

1920
219
2107
542
1378


In [5]:
######################################################
######## Box Plots of Average of Each Carrier ########
######################################################
data=[]
data.append(go.Box(
    y=finalTrebleList.difference,
    name="Treble",
    boxpoints = 'all',
))
data.append(go.Box(
    y=nonTrebleList.difference,
    name="Non-Treble",
    boxpoints = 'all',
))


# Edit the layout
layout = dict(title = "Days from Bulletins to Carrier Release Dates for Treble vs non-Treble",
              yaxis = dict(title = 'Days',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              plot_bgcolor='rgba(0,0,0,0)',
              )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig,filename='treble-vs-nonTreble')
print(finalTrebleList.difference.mean())
print(nonTrebleList.difference.mean())



22.44649446494465
26.21698113207547


In [6]:
print(finalTrebleList.iloc[:1])
print("---------------------------------------------")
print(nonTrebleList.iloc[:1])

    Carrier Manufacture Model_x Release_Date Bulletin_Level  difference  \
221    AT&T          LG     V30   2017-10-06     2017-10-01         5.0   

    Android_Level_Guessed            build  Year_Phone_Release  \
221                 False  nan nan H93120d              2017.0   

     Month_Phone_Release  checker  
221                  9.0      1.0  
---------------------------------------------
  Carrier Manufacture             Model_x Release_Date Bulletin_Level  \
0    AT&T     Alcatel  Alcatel IdealXCITE   2018-04-03     2018-02-01   

   difference Android_Level_Guessed  \
0        61.0                 False   

                                         build  Year_Phone_Release  \
0  JO.2.0.c1-00150-8909_GEN_PACK-1 nan YAMUAM0              2017.0   

   Month_Phone_Release  checker  
0                  6.0      5.0  


In [7]:
#print(finalTrebleList[finalTrebleList.Carrier=="Verizon"])
#print(nonTrebleList[nonTrebleList.Carrier=="Verizon"])

In [8]:
len(finalTrebleList.Model_x.unique())

34

In [9]:
len(nonTrebleList.Model_x.unique())

234

In [10]:
# Only consider 2019 bulletins
finalTrebleList.Bulletin_Level=finalTrebleList.Bulletin_Level.astype(str)
finalTrebleList.Bulletin_Level=finalTrebleList.Bulletin_Level.str.replace(" [0:]*","")
nonTrebleList.Bulletin_Level=nonTrebleList.Bulletin_Level.astype(str)
nonTrebleList.Bulletin_Level=nonTrebleList.Bulletin_Level.str.replace(" [0:]*","")
print(finalTrebleList.head())
print(nonTrebleList.head())

correctBulls=['2019-01-01','2019-02-01','2019-03-01','2019-04-01','2019-05-01','2019-06-01',
              '2019-07-01','2019-08-01','2019-09-01','2019-10-01','2019-11-01','2019-12-01']

filteredTreble=finalTrebleList[finalTrebleList.Bulletin_Level==correctBulls[0]]
for bull in correctBulls[1:]:
    tempdf=finalTrebleList[finalTrebleList.Bulletin_Level==bull]
    filteredTreble=filteredTreble.append(tempdf.copy(),ignore_index = True)

filteredNonTreble=nonTrebleList[nonTrebleList.Bulletin_Level==correctBulls[0]]
for bull in correctBulls[1:]:
    tempdf=nonTrebleList[nonTrebleList.Bulletin_Level==bull]
    filteredNonTreble=filteredNonTreble.append(tempdf.copy(),ignore_index = True)

    
print(len(filteredTreble))
print(len(filteredTreble.Model_x.unique()))
print(filteredTreble.Model_x.unique())
print("--------------------------------------------")
print(len(filteredNonTreble))
print(len(filteredNonTreble.Model_x.unique()))

    Carrier Manufacture Model_x Release_Date Bulletin_Level  difference  \
221    AT&T          LG     V30   2017-10-06     2017-10-01         5.0   
222    AT&T          LG     V30   2017-12-19     2017-12-01        18.0   
223    AT&T          LG     V30   2018-02-23     2018-01-01        53.0   
224    AT&T          LG     V30   2018-08-06     2018-05-01        97.0   
225    AT&T          LG     V30   2019-11-19     2019-11-01        18.0   

    Android_Level_Guessed            build  Year_Phone_Release  \
221                 False  nan nan H93120d              2017.0   
222                  True      nan nan nan              2017.0   
223                 False  nan nan H93111N              2017.0   
224                 False  nan nan H93120c              2017.0   
225                  None  nan nan H93120d              2017.0   

     Month_Phone_Release  checker  
221                  9.0      1.0  
222                  9.0      1.0  
223                  9.0      5.0  
224     

In [11]:
######################################################
######## Box Plots of Average of Each Carrier ########
######################################################
data=[]
data.append(go.Box(
    y=filteredTreble.difference,
    name="Treble",
    boxpoints = 'all',
    showlegend=False
))
data.append(go.Box(
    y=filteredNonTreble.difference,
    name="Non-Treble",
    boxpoints = 'all',
    showlegend=False
))


# Edit the layout
layout = dict(#title = "Days from 2019 Bulletins to Carrier Release for Treble vs non-Treble",
              yaxis = dict(title = 'Days',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              plot_bgcolor='rgba(0,0,0,0)',
              font=dict(size=20)
              )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig,filename='treble-vs-nonTreble-2019-buls.png')

print(filteredTreble.difference.mean())
print(filteredNonTreble.difference.mean())

pio.write_image(fig, 'treble-carrier-2019.pdf', width=1100, height=600)

21.049429657794676
30.559782608695652
